# Исследование надёжности заёмщиков — анализ банковских данных

**Описание проекта.** На основе статистики о платёжеспособности клиентов банка исследуется влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок. Даны рекомендации по построению скоринговой модели.

**Ход исследования:**

1. Обзор данных;
2. Предобработка данных;
3. Выявление зависимостей:
    1. Зависимость между количеством детей и возвратом кредита в срок;
    2. Зависимость между семейным положением и возвратом кредита в срок;
    3. Зависимость между уровнем дохода и возвратом кредита в срок.
4. Итоги исследования.

## 1. Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в data
data = pd.read_csv('C:/Users/alkon/practicum_projects/credit_project/data.csv')

In [3]:
# вывод первых 20 строк датасета
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# вывод общей информации о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**. Видим не одинаковое количество записей в столбцах, следовательно в данных есть пропуски (`total_income`, `days_employed`). Столбцы `children` (количество детей в семье) и `dob_years` выглядят нормально, с целочисленным типом данных, нужно будет проверить наличие аномальных значений. В столбце `days_employed` (общий трудовой стаж в днях) видим ошибку в данных — отрицательные значения, а также уже из первых строк заметны аномально большие значения (более 240 тыс. дней). Столбец с возрастом `dob_years` выглядит нормально. В столбце `education` заметны неявные дубликаты. В столбце `purpose` (цель получения кредита) много значений, необходимо будет обобщить их по ключевым категориям.

## 2. Предобработка данных

In [5]:
# вывод количества пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В обоих столбцах, в которых обнаружены пропуски, отсутствует равное количество значений, можно предположить, что эти столбцы связаны между собой. Возможно, имели место ошибки при вводе данных в том или ином столбце, ведущие к образованию пропусков в другом. Сокрытие информации клиентом менее вероятно, поскольку для оформления кредита необходимо предоставлять данные о трудоустройстве и доходе. 

Заменим пропуски в столбце `total_income` медианным значением по каждому типу из столбца `income_type`. Замена медианным значением в данном случае будет оптимальным решением, поскольку на уровень дохода сильно влияет тип занятости, при этом медиана устойчива к выбросам, которые могут быть характерны для переменной дохода. 

In [6]:
# цикл, который перебирает все значения в столбце 'income_type'
for t in data['income_type'].unique():
    # замена пропущенных значений в столбце 'total_income' медианным значением для соответствующего 'income_type'
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Столбец `days_employed` содержит аномальные и ошибочные значения, которые нужно обработать перед обработкой пропусков.

In [7]:
# замена отрицательных значений на абсолютные
data['days_employed'] = data['days_employed'].abs()

# вывод медианных значений кол-ва дней занятости для каждого типа заработка
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Видим аномально большие значения у безработных и пенсионеров (~1000 лет). В данном случае логичную замену представить сложно, поэтому оставим их без изменения. Пропуски заполним медианным значением по каждому типу `income_type`.

In [8]:
# цикл, который перебирает все значения в столбце 'income_type'
for t in data['income_type'].unique():
    # замена пропущенных значений в столбце 'days_employed' медианным значением для соответствующего 'income_type'
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [9]:
# проверка результата обработки пропусков
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [10]:
# вывод уникальных значений в столбце `children`
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [11]:
# подсчет количества строк с аномальными значениями
data['children'][(data['children'] == -1) | (data['children'] == 20)].count()

123

В 123 строках встречается аномальное количество детей -1 и 20. Поскольку таких строк мало от общего количество данных, удалим их.

In [12]:
# оставляем только те строки, которые не содержат значение -1 и 20
data = data[(data['children'] != -1) & (data['children'] != 20)]

# проверка обработки аномалий 
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Для удобства заменим тип данных в столбце `total_income` на целочисленный.

In [13]:
# замена типа данных на int
data['total_income'] = data['total_income'].astype(int)

Обработаем дубликаты в столбце `education`.

In [14]:
# приведение значений к нижнему регистру
data['education'] = data['education'].str.lower()

# подсчет явных дубликатов
data.duplicated().sum()

71

In [15]:
# удаление дубликатов
data = data.drop_duplicates()

Для удобства категоризируем данные. Создадим столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [16]:
def categorize_income(income):
    """
    Функция для категоризации дохода в определенные группы.

    Параметры:
    - income (float): Значение дохода, которое нужно категоризировать.

    Возвращает:
    - str: Категория дохода ('A', 'B', 'C', 'D', 'E').

    Категории:
    - 'A': Доход от 1,000,001 и выше.
    - 'B': Доход от 200,001 до 1,000,000.
    - 'C': Доход от 50,001 до 200,000.
    - 'D': Доход от 30,001 до 50,000.
    - 'E': Доход от 0 до 30,000.
    
    Примечание: Функция возвращает None, если значение дохода не попадает в ни одну из категорий.
    """
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
# применение функции `categorize_income`
data['total_income_category'] = data['total_income'].apply(categorize_income)

Перейдем к категоризации столбца `purpose`.

In [18]:
# вывод уникальных значений столбца `purpose`
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [19]:
def categorize_purpose(row):
    """
    Функция для категоризации цели займа на основе текстового описания.

    Параметры:
    - row (str): Текстовое описание цели займа.

    Возвращает:
    - str: Категория цели займа.

    Возможные категории:
    - 'операции с автомобилем': Если строка содержит ключевое слово 'автом'.
    - 'операции с недвижимостью': Если строка содержит ключевые слова 'жил' или 'недвиж'.
    - 'проведение свадьбы': Если строка содержит ключевое слово 'свад'.
    - 'получение образования': Если строка содержит ключевое слово 'образов'.

    Примечание: Если ни одно из ключевых слов не обнаружено, возвращается 'нет категории'.
    """
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
# применение функции 'categorize_purpose'
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [21]:
# вывод первый строк датасета после предобработки
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


**Вывод**. В результате предобработки пропуски в столбцах `total_income` и `days_employed` были заменены медианным значением по `income_type`. Тип данных в столбце `total_income` был приведен к целочисленному. Были выявлены и удалены неявные дубликаты по столбцу `education`. Были созданные категории для дохода и цели получения кредита (`total_income_category`, `purpose_category`).

## 3. Исследовательский анализ

### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# построение сводной таблицы для рассчета количества заемщиков и доли заемщиков по каждому значению из 'children'
children_debt = data.pivot_table(index = 'children', values='debt', aggfunc = ['count','mean']).reset_index()

# переименование столбцов
children_debt.columns = ['Кол-во детей', 'Кол-во заемщиков', 'Доля должников']

# вывод результата метода 'mean' в процентах с помощью функции.
def percent(mean):
    return "{0:.2%}".format(mean)

# применение функции к столбцу
children_debt['Доля должников'] = children_debt['Доля должников'].apply(percent)

# сортировка по проценту должников.
children_debt = children_debt.sort_values(by='Доля должников').reset_index(drop=True)

children_debt

,Кол-во детей,Кол-во заемщиков,Доля должников
0,5,9,0.00%
1,0,14091,7.54%
2,3,330,8.18%
3,1,4808,9.23%
4,2,2052,9.45%
5,4,41,9.76%


Самыми надежными заемщиками оказались клиенты, у которых нет детей. При увеличении детей увеличивается количество трат, следовательно, и понижается надежность заемщика. При этом клиентов с 3-5 детьми мы не будем принимать в расчет, поскольку количество клиентов в этих группах недостаточное для проведения анализа.

### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# создание сводной таблицы для расчета количества и доли заемщиков по каждому значению 'family_status'
family_debt = data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'mean']).reset_index()

# форматирование таблицы для лучшей читаемости
family_debt.columns = ['Семейное положение', 'Кол-во заемщиков', 'Доля должников']
family_debt['Доля должников'] = family_debt['Доля должников'].apply(percent)

# сортировка по проценту должников
family_debt = family_debt.sort_values(by='Доля должников').reset_index(drop=True)

family_debt

,Семейное положение,Кол-во заемщиков,Доля должников
0,вдовец / вдова,951,6.62%
1,в разводе,1189,7.06%
2,женат / замужем,12261,7.56%
3,гражданский брак,4134,9.31%
4,Не женат / не замужем,2796,9.76%


**Вывод:** вдовцы/вдовы чаще других категорий возвращают кредит в срок (6,62% должников). Общая зависимость состоит в том, что клиенты, которые состояли (вдовцы, разведенные) или состоят в браке, являются более ответственными заемщиками. Напротив, клиенты, не состоящие в браке (в т.ч. гражданский брак, если под ним подразумевается незарегистрированный брак/сожительство), чаще нарушают сроки выплаты кредита.

### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# построение сводной таблицы для рассчета количества заемщиков и доли заемщиков по каждому значению из 'total_income_category'.
income_debt = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count','mean']).reset_index()

# форматирование таблицы
income_debt.columns = ['Категория дохода', 'Кол-во заемщиков', 'Доля должников']
income_debt['Доля должников'] = income_debt['Доля должников'].apply(percent)

# сортировка по проценту должников
income_debt = income_debt.sort_values(by='Доля должников').reset_index(drop=True)

income_debt

,Категория дохода,Кол-во заемщиков,Доля должников
0,D,349,6.02%
1,B,5014,7.06%
2,A,25,8.00%
3,C,15921,8.50%
4,E,22,9.09%


**Вывод:** клиенты с доходом от 200 тыс. до 1 млн. рублей в месяц (категория `B`, 7.06% должников) реже допускают просрочки, чем клиенты с доходом 50–200 тыс. рублей в месяц (категория `C`, 8.50%). Мы не можем учитывать группы клиентов `D`, `A` и `E`, поскольку в них недостаточное количество наблюдений.

### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
# построение сводной таблицы для рассчета количества заемщиков и доли заемщиков по каждому значению из 'purpose_category'
purpose_debt = data.pivot_table(index = 'purpose_category', values='debt', aggfunc = ['count','mean']).reset_index()

# форматирование таблицы
purpose_debt.columns = ['Цель', 'Кол-во заемщиков', 'Доля должников']
purpose_debt['Доля должников'] = purpose_debt['Доля должников'].apply(percent)

# сортировка по проценту должников
purpose_debt = purpose_debt.sort_values(by='Доля должников').reset_index(drop=True)

purpose_debt

,Цель,Кол-во заемщиков,Доля должников
0,операции с недвижимостью,10751,7.26%
1,проведение свадьбы,2313,7.91%
2,получение образования,3988,9.25%
3,операции с автомобилем,4279,9.35%


**Вывод:** более надежные заемщики — клиенты, которые берут кредит на операции с недвижимостью (7,26% должников) и проведение свадьбы (7,91%). Менее надежными заемщиками оказались клиенты, которые берут кредит на получение образования (9,25%) и операции с автомобилем (9,35%).

## 4. Выводы

**Портрет наиболее надежного клиента**:
вдова/вдовец, не имеющий(ая) детей, с доходом 200 тыс.- 1 млн. рублей в месяц, берущий(ая) кредит на операции с недвижимостью. 

**Общие выводы:** 
- Бездетные клиенты (7.54% должников) более надежны, чем малодетные (9.30%) и многодетные (8.16%);
- Семейные клиенты более надежны: вдовцы/вдовы (6.62% должников), разведенные (7.06%), женатые/замужние (7.56%). Менее надежны клиенты, состоящие в гражданском браке (9.31%), не женатые/не замужние (9.76%);
- Клиенты с доходом от 200 тыс. до 1 млн. рублей в месяц (7.06% должников) более надежны, чем клиенты с доходом 50–200 тыс. рублей в месяц (8.50%).
- Более надежны клиенты, которые берут кредит на операции с недвижимостью (7,26% должников) и проведение свадьбы (7,91%). Менее надежны клиенты, которые берут кредит на получение образования (9,25%) и операции с автомобилем (9,35%).

**Рекомендации по построению скоринговой модели:**
- Каждый дополнительный ребенок незначительно снижает вероятность выплаты кредита в срок.
- Опыт брака повышает вероятность выплаты кредита в срок.
- Более высокий доход повышает вероятность выплаты кредита в срок.
- Цель кредита влияет на вероятность выплаты кредита в срок. Для снижения рисков рекомендуется повышенная ставка по кредитам на операции с автомобилем и кредитам на получение образования. 